# N-Puzzle

In [62]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [63]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [64]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [65]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 143017.78it/s]


array([[6, 5, 1],
       [4, 8, 3],
       [0, 7, 2]])

## A* search

In [66]:
def efficiency(quality, cost) -> float:
    return quality / cost

# Heuristic function defined as sum of manhattan distances of each tile from goal position
def hn(state: np.ndarray) -> int:
    h = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if state[i, j] != 0:
                x, y = divmod(state[i, j] - 1, PUZZLE_DIM)
                h += abs(x - i) + abs(y - j)
    return h


def solve(state: np.ndarray) -> None:
    gn = 0
    cost = 0
    visited = set()
    while not np.array_equal(state, np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))):
        actions = available_actions(state) # List possible actions
        cost += len(actions) # Add number of possible actions to cost
        gn += 1 # Increase quality by 1
        
        possible_states = []
        for action in actions:
            new_state = do_action(state, action)
            new_state_tuple = tuple(map(tuple, new_state)) # Convert to hashable type
            if new_state_tuple not in visited:
                heuristic = hn(new_state) # Calculate heuristic for current state
                fn = gn + heuristic # Compute f(n) = g(n) + h(n)
                possible_states.append((new_state, fn))
        
        state, fn = min(possible_states, key=lambda x: x[1]) # Choose state with minimum f(n)
        visited.add(tuple(map(tuple, state))) # Convert to hashable type
    eff = efficiency(gn, cost) # Compute efficiency
    print(state)
    print(f'Cost: {cost}, Quality: {gn}, Efficiency: {eff}')

solve(state)

ValueError: min() iterable argument is empty